In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
if (osp.join('..', 'py') not in sys.path): sys.path.insert(1, osp.join('..', 'py'))

In [3]:

%%time
from scrape_utils import WebScrapingUtilities
%run ../load_magic/storage.py
from ha_utils import HeaderAnalysis
from cypher_utils import CypherUtilities

wsu = WebScrapingUtilities()
uri = wsu.secrets_json['neo4j']['connect_url']
user =  wsu.secrets_json['neo4j']['username']
password = wsu.secrets_json['neo4j']['password']
s = Storage()
ha = HeaderAnalysis()
cu = CypherUtilities(uri=uri, user=user, password=password, driver=None, s=s, ha=ha)

CPU times: total: 1.98 s
Wall time: 3.32 s


In [4]:

from datetime import datetime
from urllib.parse import urlparse, parse_qs
%run ../load_magic/dataframes.py
from html_analysis import ElementAnalysis
from IPython.display import clear_output

ea = ElementAnalysis()


----

In [74]:

cypher_str = '''
    MATCH p=(start:NavigableParents)-[rels:NEXT*1..10000]->(end:NavigableParents)
    WHERE ALL(r IN rels WHERE rels[0]['file_name'] = r.file_name) AND NOT (end)-[:NEXT]->()
    RETURN
        rels[0]['file_name'] AS file_name,
        REDUCE(acc=[], n in nodes(p) | acc+[n.navigable_parent]) AS navigable_parents_list;'''
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
df = pd.DataFrame(row_objs_list)
df.navigable_parents_list.map(lambda x: len(x)).max()

2

In [75]:

cypher_str = '''
    MATCH p=(start:NavigableParents)-[rels:NEXT*1..10000]->(end:NavigableParents)
    WHERE ALL(r IN rels WHERE rels[0]['file_name'] = r.file_name) AND NOT (end)-[:NEXT]->()
    RETURN p;'''
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
rows_list = []
for row_obj in row_objs_list:
    row_dict = {k: v for k, v in row_obj['p'].nodes[0].items()}
    row_dict.update({k: v for k, v in row_obj['p'].relationships[0].items()})
    rows_list.append(row_dict)
df = pd.DataFrame(rows_list)
df.sort_values(['sequence_order', 'file_name'], ascending=[False, True])

,navigable_parent,file_name,sequence_order
30665,<div>30+ days ago</div>,Statistician_(Data_Scientist)_12_month_Roster_...,1162
30664,"<div class=""icl-u-textColor--success"">usajobs....",Statistician_(Data_Scientist)_12_month_Roster_...,1161
30663,<div>30+ days ago</div>,Statistician_(Data_Scientist)_12_month_Roster_...,1160
30662,"<div class=""icl-u-textColor--success"">usajobs....",Statistician_(Data_Scientist)_12_month_Roster_...,1159
30661,<div>30+ days ago</div>,Statistician_(Data_Scientist)_12_month_Roster_...,1158
...,...,...,...
34767,"<h2 class=""jobsearch-JobDescriptionSection-job...",temp_Data_Scientist_00ba1a22ba67ffd2.html,0000
34804,<b>Purpose Statement</b>,"temp_Senior_Data_Analyst_-_Madison,_WI_-_Indee...",0000
34860,<b>Company Description:</b>,temp_Sr._Data_Scientist_ed6a921f486e2019.html,0000
34956,"<h2 class=""item-card-title"">Job match details<...",temp_Sr._Data_Scientist_efa5ad2d82bd1fcd.html,0000


In [82]:

cypher_str = '''
    MATCH (start:NavigableParents)-[r:NEXT {sequence_order: "1162"}]->(end:NavigableParents)
    RETURN start, r, end;'''
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)

In [84]:

row_objs_list[0].keys()

dict_keys(['start', 'r', 'end'])

In [85]:

for k, v in row_objs_list[0].items():
    print(f'{k}: {v}')

start: <Node id=741851 labels=frozenset({'NavigableParents'}) properties={'navigable_parent': '<div>30+ days ago</div>'}>
r: <Relationship id=23892 nodes=(<Node id=741851 labels=frozenset({'NavigableParents'}) properties={'navigable_parent': '<div>30+ days ago</div>'}>, <Node id=741852 labels=frozenset({'NavigableParents'}) properties={'navigable_parent': 'END'}>) type='NEXT' properties={'file_name': 'Statistician_(Data_Scientist)_12_month_Roster_Direct_Hire__ff6d88fe8a9a7879.html', 'sequence_order': '1162'}>
end: <Node id=741852 labels=frozenset({'NavigableParents'}) properties={'navigable_parent': 'END'}>


In [81]:

for k, v in row_objs_list[0]['start'].items():
    print(f'{k}: {v}')

navigable_parent: <div>30+ days ago</div>


In [87]:

for k, v in row_objs_list[0]['r'].items():
    print(f'{k}: {v}')

file_name: Statistician_(Data_Scientist)_12_month_Roster_Direct_Hire__ff6d88fe8a9a7879.html
sequence_order: 1162


In [86]:

for k, v in row_objs_list[0]['end'].items():
    print(f'{k}: {v}')

navigable_parent: END


In [10]:

p = row_objs_list[0]['p']
print(type(p))
print([f'p.{fn}' for fn in dir(p) if not fn.startswith('_')])

<class 'neo4j.graph.Path'>
['p.end_node', 'p.graph', 'p.nodes', 'p.relationships', 'p.start_node']


In [43]:

r = p.relationships[0]
print(type(r))
print([f'r.{fn}' for fn in dir(r) if not fn.startswith('_')])

<class 'abc.NEXT'>
['r.end_node', 'r.get', 'r.graph', 'r.id', 'r.items', 'r.keys', 'r.nodes', 'r.start_node', 'r.type', 'r.values']


In [42]:

r

(<Relationship id=191702 nodes=(<Node id=655745 labels=frozenset({'NavigableParents'}) properties={'navigable_parent': '<b>Positions are to be filled in the Large Business &amp; International (LB&amp;I), Data Solutions division.</b>'}>, <Node id=655746 labels=frozenset({'NavigableParents'}) properties={'navigable_parent': '<b>Eight (8) positions are to be filled in any of the following locations:</b>'}>) type='NEXT' properties={'file_name': 'Statistician_(Data_Scientist)_12_month_Roster_Direct_Hire__ff6d88fe8a9a7879.html', 'sequence_order': '698'}>,)

In [44]:

for k, v in r.items():
    print(k, v)

file_name Statistician_(Data_Scientist)_12_month_Roster_Direct_Hire__ff6d88fe8a9a7879.html
sequence_order 698


In [14]:

g = p.graph
print(type(g))
print([f'g.{fn}' for fn in dir(g) if not fn.startswith('_')])

<class 'neo4j.graph.Graph'>
['g.Hydrator', 'g.nodes', 'g.relationship_type', 'g.relationships']


In [25]:

n = p.nodes[0]
print(type(n))
print([f'n.{fn}' for fn in dir(n) if not fn.startswith('_')])

<class 'neo4j.graph.Node'>
['n.get', 'n.graph', 'n.id', 'n.items', 'n.keys', 'n.labels', 'n.values']


In [36]:

for k, v in n.items():
    print(k, v)

navigable_parent <b>Positions are to be filled in the Large Business &amp; International (LB&amp;I), Data Solutions division.</b>


In [35]:

list(n.values())[0]

'<b>Positions are to be filled in the Large Business &amp; International (LB&amp;I), Data Solutions division.</b>'

In [29]:

n.labels

frozenset({'NavigableParents'})

In [28]:

n.keys()

dict_keys(['navigable_parent'])

In [ ]:

file_path = '../data/html/indeed_email.html'
page_soup = get_page_soup(file_path)
css_selector = 'body > table > tbody > tr > td > a > table > tbody > tr > td > a'
link_soups_list = page_soup.select(css_selector)
for link_soup in link_soups_list:
    url_str = link_soup['href']
    page_soup = get_page_soup(url_str)
    page_title = page_soup.find('title').string.strip()
    file_name = re.sub(r'\W+', ' ', page_title).strip().replace(' ', '_')
    jk = parse_qs(urlparse(url_str).query).get('jk', [''])[0]
    if len(jk):
        file_name = f'{jk}_{file_name}.html'
    else:
        # file_name = datetime.now().strftime('%Y%m%d%H%M%S%f') + f'_{file_name}.html'
        file_name = f'{file_name}.html'
    file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
    if not os.path.isfile(file_path):
        with open(file_path, 'w', encoding=s.encoding_type) as f:
            if verbose:
                print(f'Saving to {file_path}')
            f.write('<html><head><title>')
            f.write(page_title)
            f.write('</title></head><body>')
            row_div_list = page_soup.find_all(name='div', attrs={'class': ['jobsearch-JobComponent-description']})
            for div_soup in row_div_list:
                f.write(str(div_soup))
            f.write('</body></html>')
    clear_output(wait=True)
    file_name_id = cu.get_filename_id(file_name, verbose=verbose)
    row_div_list = page_soup.find_all(name='div', id='jobDescriptionText')
    for div_soup in row_div_list:
        child_strs_list = ea.ha.get_navigable_children(div_soup, [])
        child_tags_list = ea.ha.get_child_tags_list(child_strs_list)
        for sequence_order, (navigable_parent, header_tag) in enumerate(zip(child_strs_list, child_tags_list)):
            clear_output(wait=True)
            header_tag_id = cu.get_headertag_id(header_tag, verbose=verbose)
            header_tag_sequence_id = cu.get_headertagsequence_id(file_name_id, header_tag_id, sequence_order, verbose=verbose)
            cu.ensure_headertagsequence_filename_relationship(file_name_id, verbose=verbose)
            cu.ensure_headertagsequence_headertag_relationship(header_tag_id, verbose=verbose)

            navigable_parent_id = cu.get_navigableparent_id(navigable_parent, verbose=verbose)
            cu.ensure_headertag_navigableparent_relationship(header_tag_id, navigable_parent_id, verbose=verbose)

            navigable_parent_sequence_id = cu.get_navigableparentsequence_id(file_name_id, navigable_parent_id, sequence_order, verbose=verbose)
            cu.ensure_navigableparentsequence_filename_relationship(file_name_id, verbose=verbose)
            cu.ensure_navigableparentsequence_navigableparent_relationship(navigable_parent_id, verbose=verbose)